## Importações



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import numpy as np
import pandas as pd
import random

from funcoes import *
from funcoes import selecao_torneio_min as funcao_selecao
from funcoes import cruzamento_ponto_simples as funcao_cruzamento

In [2]:
from mofdb_client import fetch
# Fetch all mofs with void fraction >= 0.5 and <= 0.99
# Convert all isotherm loading units to mmol/g and all pressures to atmospheres

# Create a DataFrame
df = pd.DataFrame()

TAMANHO_DATAFRAME = 5

t = 0
for mof in fetch(vf_min=0, vf_max=1, loading_unit="mmol/g", pressure_unit="atm"):
    new_row = {'mof': mof.name, 'cif': mof.cif, 'void fraction': mof.void_fraction}
    # Append the new row to the DataFrame
    df = df.append(new_row, ignore_index=True)
    t += 1
    if t >= TAMANHO_DATAFRAME:
        break

## Tratamento de dados

In [3]:
df = extrair_cif(df, minimo = 5)

GlobalSymmetryFeatures:   0%|          | 0/5 [00:00<?, ?it/s]

ElementProperty:   0%|          | 0/5 [00:00<?, ?it/s]

## Código e discussão



In [4]:
print(list(df.iloc[:,2:].keys()))
print(df.index)

['spacegroup_num', 'crystal_system_int', 'n_symmetry_ops', 'Zn', 'H', 'C', 'O']
RangeIndex(start=0, stop=5, step=1)


In [5]:
print(df)
print(df.iloc[:,2:])

      mof  void fraction  spacegroup_num  crystal_system_int  n_symmetry_ops  \
0  hMOF-6       0.754903               1                   7               1   
1  hMOF-0       0.795539             160                   3               6   
2  hMOF-7       0.327134               1                   7               2   
3  hMOF-5       0.169003               1                   7               3   
4  hMOF-3       0.755062               1                   7               1   

     Zn     H     C     O  
0   4.0   6.0  24.0  13.0  
1   4.0  12.0  24.0  13.0  
2   8.0  12.0  48.0  26.0  
3  12.0  18.0  72.0  39.0  
4   4.0   6.0  24.0  13.0  
   spacegroup_num  crystal_system_int  n_symmetry_ops    Zn     H     C     O
0               1                   7               1   4.0   6.0  24.0  13.0
1             160                   3               6   4.0  12.0  24.0  13.0
2               1                   7               2   8.0  12.0  48.0  26.0
3               1                   7  

### Divisão treino-teste

In [6]:
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455
FEATURES = list(df.iloc[:,2:].keys())

TARGET = ['void fraction'] # porosidade do mof

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.reindex(FEATURES, axis=1)
y_treino = df_treino.reindex(TARGET, axis=1)
X_teste = df_teste.reindex(FEATURES, axis=1)
y_teste = df_teste.reindex(TARGET, axis=1)

In [7]:
normalizador_x = MinMaxScaler()
normalizador_y = MinMaxScaler()

normalizador_x.fit(X_treino)
normalizador_y.fit(y_treino)

X_treino = normalizador_x.transform(X_treino)
y_treino = normalizador_y.transform(y_treino)
X_teste = normalizador_x.transform(X_teste)
y_teste = normalizador_y.transform(y_teste)

In [8]:
X_treino = torch.tensor(X_treino, dtype=torch.float32)
y_treino = torch.tensor(y_treino, dtype=torch.float32)
X_teste = torch.tensor(X_teste, dtype=torch.float32)
y_teste = torch.tensor(y_teste, dtype=torch.float32)

In [9]:
class MLP(nn.Module):
    def __init__(
        self, num_dados_entrada, neuronios_c1, neuronios_c2, neuronios_c3, neuronios_c4, num_targets
    ):
        # Temos que inicializar a classe mãe
        super().__init__()

        # Definindo as camadas da rede
        self.camadas = nn.Sequential(
                    nn.Linear(num_dados_entrada, neuronios_c1),
                    nn.ReLU(),
                    nn.Linear(neuronios_c1, neuronios_c2),
                    nn.ReLU(),
                    nn.Linear(neuronios_c2, neuronios_c3),
                    nn.ReLU(),
                    nn.Linear(neuronios_c3, neuronios_c4),
                    nn.ReLU(),
                    nn.Linear(neuronios_c4, num_targets),
                )

    def forward(self, x):
        """Esse é o método que executa a rede do pytorch."""
        x = self.camadas(x)
        return x

In [10]:
NUM_DADOS_DE_ENTRADA = X_treino.shape[1]
NUM_DADOS_DE_SAIDA = y_treino.shape[1]
NEURONIOS_C1 = 150
NEURONIOS_C2 = 100
NEURONIOS_C3 = 80
NEURONIOS_C4 = 60

minha_MLP = MLP(NUM_DADOS_DE_ENTRADA, NEURONIOS_C1, NEURONIOS_C2, NEURONIOS_C3, NEURONIOS_C4, NUM_DADOS_DE_SAIDA)

In [11]:
y_prev = minha_MLP(X_treino)

In [12]:
TAXA_DE_APRENDIZADO = 0.0005

# função perda será o erro quadrático médio
fn_perda = nn.MSELoss()

# otimizador será o Adam, um tipo de descida do gradiente
otimizador = optim.Adam(minha_MLP.parameters(), lr=TAXA_DE_APRENDIZADO)

In [13]:
minha_MLP.train()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=7, out_features=150, bias=True)
    (1): ReLU()
    (2): Linear(in_features=150, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=80, bias=True)
    (5): ReLU()
    (6): Linear(in_features=80, out_features=60, bias=True)
    (7): ReLU()
    (8): Linear(in_features=60, out_features=1, bias=True)
  )
)

In [14]:
NUM_EPOCAS = 2000

y_true = y_treino

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = minha_MLP(X_treino)

    # zero grad
    otimizador.zero_grad()

    # loss
    loss = fn_perda(y_pred, y_true)

    # backpropagation
    loss.backward()

    # atualiza parâmetros
    otimizador.step()

    # mostra resultado
    print(epoca, loss.data)

0 tensor(0.4334)
1 tensor(0.4211)
2 tensor(0.4099)
3 tensor(0.3996)
4 tensor(0.3897)
5 tensor(0.3801)
6 tensor(0.3712)
7 tensor(0.3622)
8 tensor(0.3530)
9 tensor(0.3439)
10 tensor(0.3347)
11 tensor(0.3259)
12 tensor(0.3169)
13 tensor(0.3077)
14 tensor(0.2984)
15 tensor(0.2891)
16 tensor(0.2791)
17 tensor(0.2687)
18 tensor(0.2580)
19 tensor(0.2468)
20 tensor(0.2353)
21 tensor(0.2233)
22 tensor(0.2114)
23 tensor(0.1994)
24 tensor(0.1872)
25 tensor(0.1748)
26 tensor(0.1624)
27 tensor(0.1499)
28 tensor(0.1373)
29 tensor(0.1248)
30 tensor(0.1122)
31 tensor(0.0996)
32 tensor(0.0872)
33 tensor(0.0751)
34 tensor(0.0633)
35 tensor(0.0519)
36 tensor(0.0414)
37 tensor(0.0321)
38 tensor(0.0238)
39 tensor(0.0170)
40 tensor(0.0118)
41 tensor(0.0080)
42 tensor(0.0056)
43 tensor(0.0046)
44 tensor(0.0047)
45 tensor(0.0055)
46 tensor(0.0066)
47 tensor(0.0078)
48 tensor(0.0086)
49 tensor(0.0090)
50 tensor(0.0088)
51 tensor(0.0081)
52 tensor(0.0069)
53 tensor(0.0055)
54 tensor(0.0041)
55 tensor(0.0027)
56

In [15]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_treino)
    y_pred = minha_MLP(X_treino)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[0.169003] [0.16900299]
[0.754903] [0.754903]
[0.32713401] [0.32713401]
[0.795539] [0.795539]


In [16]:
minha_MLP.eval()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=7, out_features=150, bias=True)
    (1): ReLU()
    (2): Linear(in_features=150, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=80, bias=True)
    (5): ReLU()
    (6): Linear(in_features=80, out_features=60, bias=True)
    (7): ReLU()
    (8): Linear(in_features=60, out_features=1, bias=True)
  )
)

In [17]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_teste)
    y_pred = minha_MLP(X_teste)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[0.75506202] [0.75490297]


In [18]:
X_teste[0]

tensor([0., 1., 0., 0., 0., 0., 0.])

In [19]:
RMSE = mean_squared_error(y_true, y_pred, squared=False)
print(f'Loss do teste: {RMSE}')

Loss do teste: 0.00015905003690719344


## Algoritmo Genético

In [20]:
# Constantes de busca

TAMANHO_POP = 12 # quantidade de indivíduos
NUM_GERACOES = 2000 # número de gerações
CHANCE_CRUZAMENTO = 0.5 # chance de ocorrer o cruzamento entre dois indivíduos
CHANCE_MUTACAO = 0.02 # chance de ocorrer mutação em cada indivíduo durante cada geração

# Constantes de problema
QUANTIDADE_MAX_ATOMOS = 100 # quantidade de valor máximo que um gene pode assumir
NUM_ELEMENTOS = len(FEATURES) - 3 # quantidade de genes presentes em cada indivíduo
TAMANHO_PORO_DESEJADO = 0.812

In [21]:
# Funções Locais

def cria_populacao_inicial(tamanho, numero_elementos):
    return populacao_mof(tamanho, numero_elementos, QUANTIDADE_MAX_ATOMOS)

def funcao_mutacao(individuo):
    return mutacao_mof(individuo, QUANTIDADE_MAX_ATOMOS)

def lista_para_dataframe(lista):
    """ Uma função que recebe uma lista e retorna um dataframe, seguindo uma ordem de colunas
    
    Args:
        lista: uma lista qualquer
        
    Return:
        dataframe
    """
    df_lista = pd.DataFrame([lista], columns=FEATURES)
    
    return df_lista

def computa_void_fraction(individuo):
    """ Uma função que calcula o tamanho previsto para uma MOF a partir do modelo de Redes Neurais treinado.
    
    Args:
        individuo: um indivíduo válido para o problema das mofs
        
    Return:
        void_fraction previsto
    """
    individuo = lista_para_dataframe(individuo)
    individuo = normalizador_x.transform(individuo)
    individuo = torch.tensor(individuo, dtype=torch.float32)
    with torch.no_grad():
        y_pred = minha_MLP(individuo)
        y_pred = normalizador_y.inverse_transform(y_pred)
        
    return y_pred

def funcao_objetivo(individuo, tamanho_poro_desejado):
    """ Uma função que calcula o fitness de cada indivíduo do problema das mofs a partir do modelo preditivo de redes neurais
    
    Args:
        individuo: um indivíduo válido para o problema das mofs
        
    Return:
        O fitness do indivíduo
    """
    y_pred = computa_void_fraction(individuo)
    
    return abs(y_pred - tamanho_poro_desejado)

def funcao_objetivo_pop(populacao):
    """ Calcula a função objetivo para todos os membros de uma população
    
    Args:
        população: Lista com todos os indivíduos da população
        
    Return:
        Lista contendo o fitness de cada indivíduo
    """
    fitness = []
    for individuo in populacao:
        fobj = funcao_objetivo(individuo, TAMANHO_PORO_DESEJADO)
        fitness.append(fobj)
    return fitness

In [22]:
populacao = cria_populacao_inicial(TAMANHO_POP, NUM_ELEMENTOS) # cria aleatoriamente uma população inicial
hall_da_fama_individuo = []
melhor_fitness_ja_visto = float("inf")  # é assim que escrevemos infinito em python

print('População inicial:') # mostra qual foi a população criada aleatoriamente
for i, ind in enumerate(populacao):
    print('Individuo ', i+1, ': ', ind)

for _ in range(NUM_GERACOES): # loop que começa a rodar cada geração
    fitness = funcao_objetivo_pop(populacao) # cálculo da função objetivo de cada indivíduo da população
    populacao = funcao_selecao(populacao, fitness) # seleção de roleta com diferentes pesos, baseados na função fitness
    
    pais = populacao[0::2] # definição dos indivíduos que serão pais
    maes = populacao[1::2] # definição dos indivíduos que serão mães
    contador = 0 # estratégia para colocar os filhos no lugar dos pais
    for pai, mae in zip(pais, maes): # laço de repetição para pegar itens da lista de pais e mães
        if random.random() < CHANCE_CRUZAMENTO: # aplicando a possibilidade de cruzamento
            # vai acertar o cruzamento
            filho1, filho2 = funcao_cruzamento(pai, mae) # "calculando" o filho 1 e o filho 2
            populacao[contador] = filho1 # trocando o pai pelo filho 1
            populacao[contador + 1] = filho2 # trocando a mãe pelo filho 2
            
        contador = contador + 2 # atualização do contador
    
    for n in range(len(populacao)): #laço de repetição para mutação
        if random.random() <= CHANCE_MUTACAO: # chance de mutação
            individuo = populacao[n] # esxolhe o indivíduo
            populacao[n] = funcao_mutacao(individuo) # muta o indivíduo
            
    #hall da fama       
    fitness = np.array(funcao_objetivo_pop(populacao))
    determinar_melhor_fitness = min(abs(fitness-TAMANHO_PORO_DESEJADO))
    
    if determinar_melhor_fitness < melhor_fitness_ja_visto:
        posicao = np.where(determinar_melhor_fitness)[0][0]
        melhor_individuo_ja_visto = populacao[posicao]
        melhor_fitness_ja_visto = float(fitness[posicao][0])
        
        hall_da_fama_individuo.append([melhor_individuo_ja_visto,melhor_fitness_ja_visto])
    
print()
print('População final:') # mostra qual foi a população final selecionada geneticamente
for i, ind in enumerate(populacao):
    print('Individuo ', i+1, ': ', ind, computa_void_fraction(ind))
print('Hall da Fama: ',hall_da_fama_individuo)

População inicial:
Individuo  1 :  [13, 4, 4, 41, 17, 26, 98]
Individuo  2 :  [109, 4, 6, 98, 60, 56, 46]
Individuo  3 :  [74, 4, 10, 95, 68, 5, 55]
Individuo  4 :  [99, 7, 10, 28, 2, 87, 87]
Individuo  5 :  [86, 6, 4, 26, 76, 97, 8]
Individuo  6 :  [9, 4, 3, 93, 71, 64, 74]
Individuo  7 :  [68, 1, 3, 69, 5, 34, 92]
Individuo  8 :  [21, 7, 10, 49, 90, 1, 86]
Individuo  9 :  [228, 3, 11, 54, 54, 91, 53]
Individuo  10 :  [100, 5, 6, 60, 62, 51, 29]
Individuo  11 :  [54, 7, 3, 67, 91, 49, 8]
Individuo  12 :  [225, 5, 3, 21, 26, 56, 57]

População final:
Individuo  1 :  [230, 7, 12, 21, 2, 0, 0] [[0.80603645]]
Individuo  2 :  [230, 7, 12, 21, 2, 0, 0] [[0.80603645]]
Individuo  3 :  [230, 7, 12, 21, 2, 0, 0] [[0.80603645]]
Individuo  4 :  [230, 7, 12, 21, 2, 0, 0] [[0.80603645]]
Individuo  5 :  [230, 7, 12, 21, 2, 0, 0] [[0.80603645]]
Individuo  6 :  [230, 7, 12, 21, 2, 0, 0] [[0.80603645]]
Individuo  7 :  [230, 7, 12, 21, 2, 0, 0] [[0.80603645]]
Individuo  8 :  [230, 7, 12, 95, 2, 0, 0] [[

In [25]:
print(len(hall_da_fama_individuo))

132
